# Preparatory Processing and Calcs for the Heat Pump Calculator

In [7]:
import os
import pandas as pd
import numpy as np
from glob import glob
import requests

## Download TMY3 Files

In [15]:
TMY_FOLDER = 'C:/Users/Alan/Dropbox/TMY3'
TMY_URL = 'http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2005/data/tmy3/{}TYA.CSV'
#for lin in open(os.path.join(TMY_FOLDER, 'tmy_list.txt')):
#    tmy_id = lin.strip().split()[0]
#    print('Processing: {}'.format(tmy_id))
#    resp = requests.get(TMY_URL.format(tmy_id))
#    open(os.path.join(TMY_FOLDER, '{}.csv'.format(tmy_id)), 'w').write(resp.text)
